In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

transform = transforms.Compose([
#     transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            #3 224 128
            nn.Conv2d(3, 64, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            #64 112 64
            nn.Conv2d(64, 128, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            # nn.BatchNorm2d(),
            # nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            #128 56 32
            nn.Conv2d(128, 256, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            # nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            # nn.BatchNorm2d(),
            # nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            #256 28 16
            nn.Conv2d(256, 512, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            # nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            
            # nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            #512 14 8
            nn.Conv2d(512, 512, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1,stride=1),nn.ReLU(inplace=True),
            
            # nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2)
        )     #512 7 4
        self.avg_pool = nn.AdaptiveAvgPool2d(7)
        self.norm = nn.BatchNorm2d(64)
        #512 1 1
        self.fc = nn.Sequential(
            nn.Linear(7 * 7 * 512,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096,10))
        

        


        
        
        # self.fc4 = nn.Linear(1000,10)
        

    def forward(self, x):

        #print(x.size())
        features = self.conv(x)
        # x = self.norm(x)
        x = self.avg_pool(features)

        x = x.view(features.size(0), -1)

        x = self.fc(x)
        
        return x, features


net = Net()
net = net.to(device)
param = list(net.parameters())
print(len(param))
for i in param:
    print(i.shape)
#print(param[0].shape)

28
torch.Size([64, 3, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([256, 128, 3, 3])
torch.Size([256])
torch.Size([256, 256, 3, 3])
torch.Size([256])
torch.Size([512, 256, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512, 512, 3, 3])
torch.Size([512])
torch.Size([64])
torch.Size([64])
torch.Size([4096, 25088])
torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096])
torch.Size([10, 4096])
torch.Size([10])


In [ ]:
classes =  ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(),lr=0.0001)

In [ ]:
for epoch in range(25):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        #print(inputs.shape)
        #print(inputs.shape)  
        # forward + backward + optimize
        outputs,f = net(inputs)
        #print(outputs.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if(loss.item() > 1000):
            print(loss.item())
            for param in net.parameters():
                print(param.data)
        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,    50] loss: 1.900
[2,    50] loss: 1.515
[3,    50] loss: 1.354
[4,    50] loss: 1.171
[5,    50] loss: 1.102
[6,    50] loss: 0.956
[7,    50] loss: 0.831
[8,    50] loss: 0.692
[9,    50] loss: 0.593
[10,    50] loss: 0.543
[11,    50] loss: 0.427
[12,    50] loss: 0.356
[13,    50] loss: 0.201
[14,    50] loss: 0.145
[15,    50] loss: 0.098
[16,    50] loss: 0.064
[17,    50] loss: 0.050
[18,    50] loss: 0.039
[19,    50] loss: 0.040
[20,    50] loss: 0.084
[21,    50] loss: 0.059
[22,    50] loss: 0.032
[23,    50] loss: 0.072
[24,    50] loss: 0.026
[25,    50] loss: 0.016
Finished Training


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs,_ = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of airplance : 70 %
Accuracy of  bird : 56 %
Accuracy of   car : 93 %
Accuracy of   cat : 58 %
Accuracy of  deer : 59 %
Accuracy of   dog : 40 %
Accuracy of horse : 73 %
Accuracy of monkey : 72 %
Accuracy of  ship : 84 %
Accuracy of truck : 51 %
